In [1]:
#词条相似度：word2vec

# 初始化word2vec模型和词表
#1.设置word2vec模型
from gensim.models.word2vec import Word2Vec
from MyDataFrame import MyDataFrame
mdf = MyDataFrame()
df = mdf.new_DataFrame()
df2 = mdf.m_cut(df)

n_dim = 300 # 指定向量维度，大样本量时300~500较好
w2vmodel = Word2Vec(size = n_dim, min_count = 10)
w2vmodel.build_vocab(df2.fenci) # 生成词表
w2vmodel

#2.对word2vec模型进行训练
# 在评论训练集上建模（大数据集时可能会花费几分钟）
%time w2vmodel.train(df2.fenci,total_examples = w2vmodel.corpus_count, epochs = 10)# 本例消耗内存较少
# 训练完毕的模型实质
w2vmodel.save('C:/EdmsData/w2vmodel.txt')#w2v模型的保存
print(w2vmodel.wv["数据挖掘"].shape)#(300,)
w2vmodel.wv["数据挖掘"]
w2vmodel.wv.most_similar("数据挖掘")#词向量间的相似度
# 寻找对应关系
w2vmodel.wv.most_similar(['数据', '挖掘'], ['数据'], topn = 5)

C:\Tools\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache


['0', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '4', '5', '6', '7', '8', '9']


Loading model cost 1.156 seconds.
Prefix dict has been built succesfully.


Wall time: 556 ms
(300,)


[('融合', 0.9983683228492737),
 ('获取', 0.9982103109359741),
 ('海量', 0.9981887340545654),
 ('更好', 0.9981241226196289),
 ('科学', 0.9979815483093262)]

In [22]:
#文档相似度
#1.基于词袋模型计算--sklearn实现
from ToMatrix import ToMatrix
tm = ToMatrix()
res = tm.tocountvec()

from sklearn.metrics.pairwise import pairwise_distances

#print(pairwise_distances(res, metric = 'cosine'))#数组，里面是list of list
#print(pairwise_distances(res))# 默认值为euclidean，欧式空间距离

# 使用TF-IDF矩阵进行相似度计算
#pairwise_distances(tfidf[:5], metric = 'cosine')


from gensim.models.ldamodel import LdaModel
#2.gensim实现
from gensim import similarities
from MyDataFrame import MyDataFrame
mdf = MyDataFrame()
df = mdf.new_DataFrame()
df2 = mdf.m_cut(df)
filelist=[]
for i in range(len(df2)):
    filelist.append(df2['fenci'][i])
#生成文档对应的字典和bow稀疏矩阵
from gensim import corpora, models  

dictionary = corpora.Dictionary(filelist) 
corpus = [dictionary.doc2bow(text) for text in filelist]

simmtx = similarities.MatrixSimilarity(corpus)
simmtx#相似矩阵

#基于LDA计算余弦相似度
simmtx.index[:2]
# 使用gensim的LDA拟合结果进行演示
query_bow = dictionary.doc2bow(df2['fenci'][0]) # 频数向量
ldamodel = LdaModel(corpus, id2word = dictionary, num_topics = 20, passes = 20)#拟合的好，最大值接近1
lda_vec = ldamodel[query_bow] # 转换为lda模型下的向量
sims = simmtx[lda_vec] # 进行矩阵内向量和所提供向量的余弦相似度查询
sims = sorted(enumerate(sims), key=lambda item: -item[1])
sims

['0', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '4', '5', '6', '7', '8', '9']


[(13, 0.014236408),
 (0, 0.0095406994),
 (1, 0.0095406994),
 (2, 0.0),
 (3, 0.0),
 (4, 0.0),
 (5, 0.0),
 (6, 0.0),
 (7, 0.0),
 (8, 0.0),
 (9, 0.0),
 (10, 0.0),
 (11, 0.0),
 (12, 0.0),
 (14, 0.0),
 (15, 0.0),
 (16, 0.0),
 (17, 0.0),
 (18, 0.0),
 (19, 0.0),
 (20, 0.0),
 (21, 0.0),
 (22, 0.0),
 (23, 0.0),
 (24, 0.0),
 (25, 0.0),
 (26, 0.0),
 (27, 0.0),
 (28, 0.0),
 (29, 0.0),
 (30, 0.0)]

In [10]:
#doc2vec

import jieba 
import gensim
from gensim.models import doc2vec

from MyDataFrame import MyDataFrame
mdf = MyDataFrame()
df = mdf.new_DataFrame()
df2 = mdf.m_cut(df)

def m_doc(doclist):
    reslist = []
    for i, doc in enumerate(doclist):
        reslist.append(doc2vec.TaggedDocument(doc, [i]))
    return reslist

corp = m_doc(df2.fenci)
corp[:2]

d2vmodel = gensim.models.Doc2Vec(vector_size = 300, window = 20, min_count = 5)
%time d2vmodel.build_vocab(corp) #52s
d2vmodel.wv.vocab #dict

# 将新文本转换为相应维度空间下的向量
newvec = d2vmodel.infer_vector(df2.fenci[0])
simfile = d2vmodel.docvecs.most_similar([newvec], topn = 10)
for i in range(len(simfile)):
    print(simfile[i])
    a,b = simfile[i]
    print(df.filename[i])

['2009_2016年云南省医疗机构消毒灭菌效果趋势评价_李建云_周晓梅', '_一带一路_背景下_国际投资_课_省略_路径探析_以广西民族师范学院为例_郑国富', '三级医院医疗检查结果互认实施现状及影响因素分析_肖晓华_廖惠_梁恒斌_潘振威_苏', '不同等级医院医疗检查结果互认标准研究_肖晓华_谢欣睿_梁恒斌_廖惠_潘振威_苏茹', '乡村振兴战略下环巢湖休闲农业发展研究_沈东生', '二三级医院影像学检查结果互认现状及影响因素研究_肖晓华_苏茹茹_潘振威_廖惠_梁', '关于建筑工程管理的影响因素分析与对策探讨_王君霞_战丙利', '关于彰武县做好互换并地后续工作的建议_张凤华', '关于深化河北省农业水价综合改革的对策建议_马素英', '农村非正规金融发展与农村多维贫困_基于面板门槛模型的研究_吴君娴_黄永兴', '医师多点执业对医院人力资源管理的影响_张妍_李吉', '协同创新优化现代高效农业组织结构_王瑜', '县级公立医院患者体验的影响因素研究_朱锦_胡丹_陈家应', '双创背景下大学生农业创业融资问题探析_王涛_白林_齐骥霆', '吉林省农村金融发展状况及对策研究_孙铁柱', '国家重点研发计划_畜禽_专项立项特征研究_姜玮', '地方高校工科专业课程教学模式改革_省略_临沂大学_大气污染控制工程_为例_马宏卿', '基于大数据的商业智能在电商数据分析中的应用_钱丹丹', '基于数据挖掘技术的档案馆信息快速分析算法研究_甘璐', '基于数据挖掘的电影票房分析_席稼玮', '基于机器视觉和机器学习技术的鸡胴体质量自动分级方法研究_戚超', '工程教育专业认证下自动化专业实践教学体系的构建_黄宜庆_陆华才', '工程造价指数与工程造价动态管理刍议_孙静', '广西综合医院新生儿病房分级建设和能力建设现状调查_胡琴燕_韦琴_黄晓波_杨少丽_', '应用型高校金融学课程教学改革研究_邓旭霞', '建筑工程施工管理及创新技术的应用研究_侯帅', '建筑工程施工管理的进度管理与控制_吕萌', '建筑结构设计中的抗震结构设计理念_马玉洁', '建筑门窗用硅烷改性聚醚胶的性能与应用研究_杨苏邯_陈洋庆_陈建军_龙飞_蒋金博_', '我国医疗检查结果互认制度实施现状_肖晓华_梁恒斌_谢欣睿_苏茹茹_孙刚', '数据挖掘技术在语音识别中的应用_许元洪_郭琼', '数

In [4]:
#文档聚类
# 进行聚类分析
from sklearn.cluster import KMeans  

from ToMatrix import ToMatrix

tm = ToMatrix()
tfidf = tm.totfidf()

clf = KMeans(n_clusters = 5)#聚为5类
s = clf.fit(tfidf)  #通过tfidf弱化控件距离问题
print(s)
print(clf.cluster_centers_)
print(clf.cluster_centers_.shape)
print(clf.labels_)

from MyDataFrame import MyDataFrame
mdf = MyDataFrame()
df = mdf.new_DataFrame()

df['clsres'] = clf.labels_
df.head(30)

df.sort_values('clsres').clsres

import pandas as pd
import jieba
dfgrp = df.groupby('clsres')
dfcls = dfgrp.agg(sum) # 只有字符串列的情况下，sum函数自动转为合并字符串
stoplist = list(pd.read_csv('C:/TMdata/stopwords.txt',names=['w'],sep='aaa',encoding='utf-8',engine='python').w)
def m_cut(intxt):
    return [ w for w in jieba.cut(intxt) 
            if w not in stoplist and len(w) > 1 ] 
    
cuttxt = lambda x: " ".join(m_cut(x)) 
dfclsres = dfcls.content.apply(cuttxt) 
dfclsres
# 列出关键词以刻画类别特征
import jieba.analyse as ana
ana.set_stop_words('C:/TMdata/stopwords.txt')
for item in dfclsres:
    print(ana.extract_tags(item, topK = 10))

['2009_2016年云南省医疗机构消毒灭菌效果趋势评价_李建云_周晓梅', '_一带一路_背景下_国际投资_课_省略_路径探析_以广西民族师范学院为例_郑国富', '三级医院医疗检查结果互认实施现状及影响因素分析_肖晓华_廖惠_梁恒斌_潘振威_苏', '不同等级医院医疗检查结果互认标准研究_肖晓华_谢欣睿_梁恒斌_廖惠_潘振威_苏茹', '乡村振兴战略下环巢湖休闲农业发展研究_沈东生', '二三级医院影像学检查结果互认现状及影响因素研究_肖晓华_苏茹茹_潘振威_廖惠_梁', '关于建筑工程管理的影响因素分析与对策探讨_王君霞_战丙利', '关于彰武县做好互换并地后续工作的建议_张凤华', '关于深化河北省农业水价综合改革的对策建议_马素英', '农村非正规金融发展与农村多维贫困_基于面板门槛模型的研究_吴君娴_黄永兴', '医师多点执业对医院人力资源管理的影响_张妍_李吉', '协同创新优化现代高效农业组织结构_王瑜', '县级公立医院患者体验的影响因素研究_朱锦_胡丹_陈家应', '双创背景下大学生农业创业融资问题探析_王涛_白林_齐骥霆', '吉林省农村金融发展状况及对策研究_孙铁柱', '国家重点研发计划_畜禽_专项立项特征研究_姜玮', '地方高校工科专业课程教学模式改革_省略_临沂大学_大气污染控制工程_为例_马宏卿', '基于大数据的商业智能在电商数据分析中的应用_钱丹丹', '基于数据挖掘技术的档案馆信息快速分析算法研究_甘璐', '基于数据挖掘的电影票房分析_席稼玮', '基于机器视觉和机器学习技术的鸡胴体质量自动分级方法研究_戚超', '工程教育专业认证下自动化专业实践教学体系的构建_黄宜庆_陆华才', '工程造价指数与工程造价动态管理刍议_孙静', '广西综合医院新生儿病房分级建设和能力建设现状调查_胡琴燕_韦琴_黄晓波_杨少丽_', '应用型高校金融学课程教学改革研究_邓旭霞', '建筑工程施工管理及创新技术的应用研究_侯帅', '建筑工程施工管理的进度管理与控制_吕萌', '建筑结构设计中的抗震结构设计理念_马玉洁', '建筑门窗用硅烷改性聚醚胶的性能与应用研究_杨苏邯_陈洋庆_陈建军_龙飞_蒋金博_', '我国医疗检查结果互认制度实施现状_肖晓华_梁恒斌_谢欣睿_苏茹茹_孙刚', '数据挖掘技术在语音识别中的应用_许元洪_郭琼', '数

In [31]:
from gensim.models.ldamodel import LdaModel
#2.gensim实现
from gensim import similarities
from MyDataFrame import MyDataFrame
mdf = MyDataFrame()
df = mdf.new_DataFrame()
df2 = mdf.m_cut(df)

filelist=[]
for i in range(len(df2)):
    filelist.append(df2['fenci'][i])
#生成文档对应的字典和bow稀疏矩阵
from gensim import corpora, models  

dictionary = corpora.Dictionary(filelist)  
corpus = [dictionary.doc2bow(text) for text in filelist] # 仍为list in list  

tfidf_model = models.TfidfModel(corpus)#建立TF-IDF模型
corpus_tfidf = tfidf_model[corpus]#对所需文档计算TF-IDF结果

simmtx = similarities.MatrixSimilarity(corpus_tfidf)
simmtx#相似矩阵

#基于LDA计算余弦相似度

# 使用gensim的LDA拟合结果进行演示


dictionary1 = corpora.Dictionary(df2['fenci'][0])  
cor1 = dictionary1.doc2bow(df2['fenci'][0])
tfidf_model1 = models.TfidfModel(cor1)#建立TF-IDF模型
corpus_tfidf1 = tfidf_model1[cor1]#对所需文档计算TF-IDF结果
query_bow = corpus_tfidf1 # 频数向量

ldamodel = models.ldamodel.LdaModel.load('C:/EdmsData/ldamodel.txt')#拟合的好，最大值接近1

lda_vec = ldamodel[query_bow] # 转换为lda模型下的向量
sims = simmtx[lda_vec] # 进行矩阵内向量和所提供向量的余弦相似度查询
sims = sorted(enumerate(sims), key=lambda item: -item[1])
sims

['0', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '4', '5', '6', '7', '8', '9']


TypeError: doc2bow expects an array of unicode tokens on input, not a single string

In [15]:
#文档相似度
#1.基于词袋模型计算--sklearn实现
from ToMatrix import ToMatrix
tm = ToMatrix()
res = tm.tocountvec()#词频矩阵

from sklearn.metrics.pairwise import pairwise_distances

print(pairwise_distances(res, metric = 'cosine'))#数组，里面是list of list
#print(pairwise_distances(res))# 默认值为euclidean，欧式空间距离

# 使用TF-IDF矩阵进行相似度计算
#pairwise_distances(tfidf[:5], metric = 'cosine')


[[  0.00000000e+00   4.44089210e-16   7.75761547e-01   6.31403578e-01
    7.82242251e-01   5.41794277e-01   8.10176159e-01   8.17638283e-01
    6.86010196e-01   7.15065928e-01   6.19152106e-01   5.97668380e-01
    6.25136199e-01   6.85830437e-01   5.93038821e-01   7.54842755e-01
    5.22272650e-01   7.93444270e-01   4.83051942e-01   7.41869796e-01
    3.83251731e-01   7.39684761e-01   7.37300452e-01   8.14384436e-01
    8.08798370e-01   8.35949075e-01   7.13782581e-01   8.60194455e-01
    7.94583336e-01   7.52978292e-01   6.76365505e-01]
 [  4.44089210e-16   0.00000000e+00   7.75761547e-01   6.31403578e-01
    7.82242251e-01   5.41794277e-01   8.10176159e-01   8.17638283e-01
    6.86010196e-01   7.15065928e-01   6.19152106e-01   5.97668380e-01
    6.25136199e-01   6.85830437e-01   5.93038821e-01   7.54842755e-01
    5.22272650e-01   7.93444270e-01   4.83051942e-01   7.41869796e-01
    3.83251731e-01   7.39684761e-01   7.37300452e-01   8.14384436e-01
    8.08798370e-01   8.35949075e-01 

In [28]:
#只有两个文档进行lda模型拟合并查找相似文档
from gensim.models.ldamodel import LdaModel
#2.gensim实现
from gensim import similarities
from MyDataFrame import MyDataFrame
mdf = MyDataFrame()
df = mdf.new_DataFrame()
df2 = mdf.m_cut(df)

filelist=[df2.fenci[0],df2.fenci[1]]

#生成文档对应的字典和bow稀疏矩阵
from gensim import corpora, models  

dictionary = corpora.Dictionary(filelist)  
corpus = [dictionary.doc2bow(text) for text in filelist] # 仍为list in list  

simmtx = similarities.MatrixSimilarity(corpus)
simmtx#相似矩阵

#基于LDA计算余弦相似度

# 使用gensim的LDA拟合结果进行演示
query_bow = dictionary.doc2bow(df2['fenci'][0]) # 频数向量

ldamodel = models.ldamodel.LdaModel.load('C:/EdmsData/ldamodel.txt')#拟合的好，最大值接近1

lda_vec = ldamodel[query_bow] # 转换为lda模型下的向量
sims = simmtx[lda_vec] # 进行矩阵内向量和所提供向量的余弦相似度查询
sims = sorted(enumerate(sims), key=lambda item: -item[1])
sims

['0', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '4', '5', '6', '7', '8', '9']


[(0, 0.0095402654), (1, 0.0095402654)]

['0', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '4', '5', '6', '7', '8', '9']
<class 'list'>
